In [3]:
# Importul bibliotecilor necesare pentru inregistrare audio, manipulare fisiere si utilizarea SDK-ului ElevenLabs
import os
import sys
import wave
import keyboard
import pyaudio
from pydub import AudioSegment
from dotenv import load_dotenv
from io import BytesIO
from elevenlabs.client import ElevenLabs

# ---------------------------- Configuratie ----------------------------

# Incarca variabilele de mediu (inclusiv cheia API) din fisierul .env
load_dotenv()
API_KEY = os.getenv("ELEVENLABS_API_KEY")

# Verifica existenta cheii API in fisierul .env
if not API_KEY:
    print("Cheia API ElevenLabs lipseste din fisierul .env.")
    sys.exit(1)

# Initializeaza clientul ElevenLabs folosind cheia din .env
client = ElevenLabs(api_key=API_KEY)

# ---------------------------- Inregistrare Audio ----------------------------

def record_audio(output_file="recorded_audio.mp3"):
    """
    Inregistreaza sunetul de la microfon pana cand utilizatorul apasa tasta 'q'.
    Salveaza inregistrarea in format MP3.
    """
    CHUNK = 1024               # Numarul de frame-uri pe buffer
    FORMAT = pyaudio.paInt16   # Formatul de esantionare audio
    CHANNELS = 1               # Inregistrare mono
    RATE = 16000               # Frecventa de esantionare

    # Configureaza fluxul de inregistrare audio
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

    print("Se inregistreaza... Apasa 'q' pentru a opri.")
    frames = []

    # Inregistreaza pana cand se apasa tasta 'q'
    while True:
        data = stream.read(CHUNK, exception_on_overflow=False)
        frames.append(data)
        if keyboard.is_pressed("q"):
            print("Inregistrarea s-a oprit.")
            break

    # Opreste fluxul si inchide sesiunea PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Salveaza inregistrarea in format WAV temporar, apoi converteste in MP3
    temp_wav = "temp_audio.wav"
    wf = wave.open(temp_wav, "wb")
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b"".join(frames))
    wf.close()

    # Conversie WAV in MP3
    audio = AudioSegment.from_file(temp_wav)
    audio.export(output_file, format="mp3")
    os.remove(temp_wav)
    print(f"Inregistrarea a fost salvata in {output_file}")
    return output_file

# ---------------------------- Conversie Video ----------------------------

def convert_mp4_to_mp3(input_file, output_file="converted_audio.mp3"):
    """
    Converteste un fisier MP4 (video) intr-un fisier MP3 (audio).
    """
    try:
        audio = AudioSegment.from_file(input_file)
        audio.export(output_file, format="mp3")
        print(f"S-a convertit {input_file} in {output_file}")
        return output_file
    except Exception as e:
        print(f"Eroare la conversia MP4 in MP3: {e}")
        return None

# ---------------------------- Transcriere Audio ----------------------------

def transcribe_audio(audio_file, language_code="eng"):
    """
    Transcrie un fisier audio folosind ElevenLabs SDK si modelul scribe_v1.
    Accepta fisiere MP3 si returneaza transcriptul in text.
    """
    try:
        with open(audio_file, "rb") as f:
            audio_data = BytesIO(f.read())

        # Apeleaza metoda de transcriere cu parametri suplimentari (diarizare si detectie evenimente audio)
        transcription = client.speech_to_text.convert(
            file=audio_data,
            model_id="scribe_v1",
            tag_audio_events=False,
            language_code=language_code,
            diarize=False
        )

        return transcription.text.strip()

    except Exception as e:
        print(f"Eroare la transcriere: {e}")
        return None

# ---------------------------- Salvare Transcript ----------------------------

def save_transcription(text, base_dir="Transcripts"):
    """
    Salveaza textul transcris intr-un fisier .txt in directorul 'TranscriptsAzure'.
    Creeaza un fisier unic pentru fiecare transcriere.
    """
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    counter = 1
    while True:
        filename = f"transcript_{counter}.txt"
        filepath = os.path.join(base_dir, filename)
        if not os.path.exists(filepath):
            break
        counter += 1

    with open(filepath, "w", encoding="utf-8") as f:
        f.write(text)
    print(f"Transcrierea a fost salvata in {filepath}")

# ---------------------------- Program Principal ----------------------------

def main():
    """
    Interfata de comanda pentru a selecta una dintre optiunile:
    1. Inregistrare audio
    2. Transcriere MP3
    3. Convertire MP4 in MP3 si transcriere
    """
    print("Alege o optiune:")
    print("1. Inregistreaza audio nou")
    print("2. Transcrie fisier MP3 existent")
    print("3. Converteste MP4 in MP3 si transcrie")
    choice = input("Introdu 1, 2 sau 3: ")

    if choice == "1":
        audio_file = record_audio()
        if audio_file:
            lang = input("Introdu codul limbii (ex: eng pentru engleza, ron pentru romana): ").strip() or "eng"
            text = transcribe_audio(audio_file, language_code=lang)
            if text:
                save_transcription(text)

    elif choice == "2":
        file_path = input("Introdu calea catre fisierul MP3: ").strip()
        if os.path.exists(file_path) and file_path.lower().endswith(".mp3"):
            lang = input("Introdu codul limbii (ex: eng pentru engleza, ron pentru romana): ").strip() or "eng"
            text = transcribe_audio(file_path, language_code=lang)
            if text:
                save_transcription(text)
        else:
            print("Fisierul nu a fost gasit sau nu este un fisier MP3.")

    elif choice == "3":
        file_path = input("Introdu calea catre fisierul MP4: ").strip()
        if os.path.exists(file_path) and file_path.lower().endswith(".mp4"):
            mp3_file = convert_mp4_to_mp3(file_path)
            if mp3_file:
                lang = input("Introdu codul limbii (ex: eng pentru engleza, ron pentru romana): ").strip() or "eng"
                text = transcribe_audio(mp3_file, language_code=lang)
                if text:
                    save_transcription(text)
        else:
            print("Fisierul nu a fost gasit sau nu este un fisier MP4.")

    else:
        print("Optiune invalida!")

# Executa programul principal
if __name__ == "__main__":
    main()


Alege o optiune:
1. Inregistreaza audio nou
2. Transcrie fisier MP3 existent
3. Converteste MP4 in MP3 si transcrie
Transcrierea a fost salvata in Transcripts1\transcript_1.txt
